In [1]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import (
    EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3,
    EfficientNetB4, EfficientNetB5, EfficientNetB6, EfficientNetB7,
    EfficientNetV2B0, EfficientNetV2B1, EfficientNetV2B2, EfficientNetV2B3,
    EfficientNetV2S, EfficientNetV2M, EfficientNetV2L
)
from tensorflow.keras.applications.efficientnet import preprocess_input as effnetv1_preprocess
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input as effnetv2_preprocess
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from IPython.display import display

# Set seeds for reproducibility
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Constants
train_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/train'
test_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/test'
BATCH_SIZE = 32
EPOCHS = 64
IMG_SIZE = (224, 224)

# Data Generators
def get_generators(preprocess_func):
    train_gen = ImageDataGenerator(
        preprocessing_function=preprocess_func,
        horizontal_flip=True,
        zoom_range=0.2,
        rotation_range=15
    ).flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        seed=SEED
    )

    test_gen = ImageDataGenerator(
        preprocessing_function=preprocess_func
    ).flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False
    )
    return train_gen, test_gen

# Build classifier
def build_model(base_model):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(1, activation='sigmoid')(x)
    return Model(inputs=base_model.input, outputs=output)

# EfficientNet Models
models = {
    'EfficientNetB0': (EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv1_preprocess),
    'EfficientNetB1': (EfficientNetB1(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv1_preprocess),
    'EfficientNetB2': (EfficientNetB2(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv1_preprocess),
    'EfficientNetB3': (EfficientNetB3(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv1_preprocess),
    'EfficientNetB4': (EfficientNetB4(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv1_preprocess),
    'EfficientNetB5': (EfficientNetB5(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv1_preprocess),
    'EfficientNetB6': (EfficientNetB6(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv1_preprocess),
    'EfficientNetB7': (EfficientNetB7(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv1_preprocess),
    'EfficientNetV2B0': (EfficientNetV2B0(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv2_preprocess),
    'EfficientNetV2B1': (EfficientNetV2B1(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv2_preprocess),
    'EfficientNetV2B2': (EfficientNetV2B2(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv2_preprocess),
    'EfficientNetV2B3': (EfficientNetV2B3(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv2_preprocess),
    'EfficientNetV2S': (EfficientNetV2S(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv2_preprocess),
    'EfficientNetV2M': (EfficientNetV2M(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv2_preprocess),
    'EfficientNetV2L': (EfficientNetV2L(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv2_preprocess),
}

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.2)

# Result storage
report_rows = []

# Training and Evaluation Loop
for model_name, (base_model, preprocess_func) in models.items():
    print(f"\n🔧 Training model: {model_name}")
    train_gen, test_gen = get_generators(preprocess_func)
    model = build_model(base_model)

    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(
        train_gen,
        epochs=EPOCHS,
        validation_data=test_gen,
        callbacks=[early_stop, reduce_lr],
        verbose=2
    )

    predictions = model.predict(test_gen, verbose=0)
    predicted_labels = (predictions > 0.5).astype(int).ravel()
    true_labels = test_gen.classes

    accuracy = accuracy_score(true_labels, predicted_labels)
    report = classification_report(true_labels, predicted_labels, digits=6, output_dict=True)

    row = {
        'Model': model_name,
        'Accuracy': accuracy,
        'Macro Precision': report['macro avg']['precision'],
        'Macro Recall': report['macro avg']['recall'],
        'Macro F1': report['macro avg']['f1-score'],
        'Weighted Precision': report['weighted avg']['precision'],
        'Weighted Recall': report['weighted avg']['recall'],
        'Weighted F1': report['weighted avg']['f1-score'],
    }
    report_rows.append(row)

# Save and display CSV
df_report = pd.DataFrame(report_rows)
csv_path = '/kaggle/working/classification_summary.csv'
df_report.to_csv(csv_path, index=False)
print(f"✅ Saved summary to {csv_path}")
print("📊 Classification Summary:")
display(df_report)


2025-07-17 14:38:41.687593: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752763121.918960      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752763121.988213      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1752763136.529570      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
27018416/27018416 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
31790344/31790344 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
71686520/71686520 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
115263384/115263384 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
165234480/165234480 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
258076736/258076736 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step
24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
28456008/28456008 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
35839040/35839040 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
52606240/52606240 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
82420632/82420632 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
214201816/214201816 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
473176280/473176280 ━━━━━━━━━━━━━━━━━━━━ 12s 0us/step

🔧 Training model: EfficientNetB0
Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/64


I0000 00:00:1752763251.124712      57 service.cc:148] XLA service 0x7e2594003c10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752763251.125602      57 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1752763252.995363      57 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1752763261.509977      57 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/64
163/163 - 197s - 1s/step - accuracy: 0.9080 - loss: 0.2164 - val_accuracy: 0.8734 - val_loss: 0.2904 - learning_rate: 0.0010
Epoch 2/64
163/163 - 94s - 574ms/step - accuracy: 0.9358 - loss: 0.1600 - val_accuracy: 0.8974 - val_loss: 0.2579 - learning_rate: 0.0010
Epoch 3/64
163/163 - 92s - 567ms/step - accuracy: 0.9417 - loss: 0.1499 - val_accuracy: 0.8974 - val_loss: 0.2411 - learning_rate: 0.0010
Epoch 4/64
163/163 - 93s - 569ms/step - accuracy: 0.9467 - loss: 0.1399 - val_accuracy: 0.9006 - val_loss: 0.2464 - learning_rate: 0.0010
Epoch 5/64
163/163 - 93s - 570ms/step - accuracy: 0.9406 - loss: 0.1418 - val_accuracy: 0.8766 - val_loss: 0.2685 - learning_rate: 0.0010
Epoch 6/64
163/163 - 93s - 568ms/step - accuracy: 0.9490 - loss: 0.1267 - val_accuracy: 0.9054 - val_loss: 0.2440 - learning_rate: 2.0000e-04

🔧 Training model: EfficientNetV2L
Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/64
163/163 - 241s - 1s/step - accuracy: 0.9020 - loss: 0.2263 - val_accuracy: 0.8462 - val_loss: 0.3538 - learning_rate: 0.0010
Epoch 2/64
163/163 - 100s - 612ms/step - accuracy: 0.9344 - loss: 0.1569 - val_accuracy: 0.8814 - val_loss: 0.3141 - learning_rate: 0.0010
Epoch 3/64
163/163 - 97s - 593ms/step - accuracy: 0.9390 - loss: 0.1545 - val_accuracy: 0.8670 - val_loss: 0.3310 - learning_rate: 0.0010
Epoch 4/64
163/163 - 94s - 576ms/step - accuracy: 0.9373 - loss: 0.1523 - val_accuracy: 0.8510 - val_loss: 0.3799 - learning_rate: 0.0010
Epoch 5/64
163/163 - 95s - 581ms/step - accuracy: 0.9475 - loss: 0.1337 - val_accuracy: 0.8862 - val_loss: 0.3027 - learning_rate: 2.0000e-04
Epoch 6/64
163/163 - 93s - 572ms/step - accuracy: 0.9498 - loss: 0.1257 - val_accuracy: 0.8782 - val_loss: 0.3088 - learning_rate: 2.0000e-04
Epoch 7/64
163/163 - 94s - 577ms/step - accuracy: 0.9532 - loss: 0.1205 - val_accuracy: 0.8718 - val_loss: 0.3085 - learning_rate: 2.0000e-04
Epoch 8/64
163/163 - 94

,Model,Accuracy,Macro Precision,Macro Recall,Macro F1,Weighted Precision,Weighted Recall,Weighted F1
0,EfficientNetB0,0.907051,0.899900,0.902564,0.901188,0.907438,0.907051,0.907206
1,EfficientNetB1,0.934295,0.928506,0.932051,0.930208,0.934691,0.934295,0.934430
2,EfficientNetB2,0.881410,0.889906,0.856410,0.868429,0.884290,0.881410,0.878761
3,EfficientNetB3,0.923077,0.920576,0.914530,0.917369,0.922844,0.923077,0.922798
4,EfficientNetB4,0.916667,0.915154,0.905983,0.910154,0.916465,0.916667,0.916201
5,EfficientNetB5,0.910256,0.901510,0.910256,0.905355,0.912170,0.910256,0.910740
6,EfficientNetB6,0.921474,0.918480,0.913248,0.915724,0.921228,0.921474,0.921227
7,EfficientNetB7,0.894231,0.897663,0.875214,0.884158,0.895138,0.894231,0.892698
8,EfficientNetV2B0,0.910256,0.926607,0.885470,0.899982,0.916153,0.910256,0.907996
9,EfficientNetV2B1,0.899038,0.890189,0.897009,0.893276,0.900491,0.899038,0.899476
